## Expert Knowledge Worker

### A question answering Assistant that is an expert knowledge worker. To be used by employees of Insurellm, an Insurance Tech company

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import glob
from pathlib import Path
import gradio as gr

In [ ]:
load_dotenv(override=True)

api_key = os.getenv('GROQ_API_KEY')
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

if api_key:
    print(f"API key is Found starting with {api_key[:4]} and Good to Go!")
else:
    print("There is an issue with API key.")

MODEL = "openai/gpt-oss-120b"
groq = OpenAI(base_url=GROQ_BASE_URL, api_key=api_key)

In [ ]:
knowledge = {}

filenames = glob.glob("knowledge-base/employees/*")

for filename in filenames:
    name = Path(filename).stem.split(' ')[-1]
    with open(filename, mode="r", encoding="utf-8") as f:
        knowledge[name.lower()] = f.read()

In [ ]:
filenames = glob.glob("knowledge-base/products/*")

for filename in filenames:
    name = Path(filename).stem
    with open(filename, mode="r", encoding="utf-8") as f:
        knowledge[name.lower()] = f.read()

In [ ]:
SYSTEM_PREFIX = """
You represent Insurellm, the Insurance Tech company.
You are an expert in answering questions about Insurellm; its employees and its products.
You are provided with additional context that might be relevant to the user's question.
Give brief, accurate answers. If you don't know the answer, say so.

Relevant context:
"""

In [ ]:
def get_relevant_context(message):
    text = ''.join(ch for ch in message if ch.isalpha() or ch.isspace())
    words = text.lower().split()
    return [knowledge[word] for word in words if word in knowledge]

In [ ]:
def additional_content(message):
    relevant_context = get_relevant_context(message)
    if not relevant_context:
        result = "There is no additional context relevant to the user question."
    else:
        result = "The following additional context might be relevant in answering the user's question:\n\n"
        result += "\n\n".join(relevant_context)
    
    return result

In [ ]:
def chat(message, history):
    system_message = SYSTEM_PREFIX + additional_content(message)
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = groq.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

In [ ]:
view = gr.ChatInterface(fn=chat).launch(inbrowser=True)
